In [1]:
import umap
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import textwrap
import scipy

## Loading data

In [2]:
data = pd.read_csv('Table_S2.csv', index_col=0)
data['log_mass_change'] = np.log10(data['mass_change'])
data.head()

,Al,Co,Cr,Cu,Fe,Hf,Mn,Mo,Nb,Ni,...,Zr,T (C),freeze_range,literature,mass_change,max_phase,max_phase_fraction,solidus,composition,log_mass_change
2,0,30,0,0,15,0,0,0,0,5,...,0,1000,365.999017,False,40.016338,BCC_B2#2,0.512862,866.852632,Co30 V30 Si20 Fe15 Ni5,1.602237
7,0,0,0,40,10,0,0,0,0,20,...,0,1000,99.111614,False,11.863285,CU6NISI3,0.449714,1157.259027,Cu40 Si30 Ni20 Fe10,1.074205
8,0,10,15,45,0,0,0,0,25,0,...,0,1000,466.998652,False,21.987382,FCC_L12,0.453770,1088.657416,Cu45 Nb25 Cr15 Co10 Ta5,1.342174
10,0,0,0,0,10,0,0,0,0,30,...,0,1000,392.470429,False,18.670031,CO2SI_C23,0.600000,1204.657615,Si35 Ni30 V25 Fe10,1.271145
12,0,10,20,0,0,0,30,0,0,40,...,0,1000,42.118970,False,3.248910,FCC_L12,1.000000,1185.034286,Ni40 Mn30 Cr20 Co10,0.511738


In [3]:
literature_data = pd.read_csv('Table_S1.csv')
literature_data['log_mass_change'] = np.log10(literature_data['mass_change'])
literature_data['literature'] = np.ones(literature_data.shape[0], dtype=bool)
literature_data.head()

,no.material,composition,T (C),Al,Co,Cr,Cu,Fe,Hf,Mn,...,V,W,Zr,mass_change,trend if not parabolic,category,if extrapolated,citation,log_mass_change,literature
0,1,20Mo–20W–20Al–20Cr–20Ti,1000,20.0,0.0,20.0,0.0,0.0,0.0,0.0,...,0.0,20.0,0.0,6.480000,NaN,HEA,NaN,"B. Gorr et al., “High-Temperature Oxidation Be...",0.811575,True
1,1,20Mo–20W–20Al–20Cr–20Ti,1100,20.0,0.0,20.0,0.0,0.0,0.0,0.0,...,0.0,20.0,0.0,5.504587,NaN,HEA,NaN,"B. Gorr et al., “High-Temperature Oxidation Be...",0.740725,True
2,2,20Nb-20Mo-20Cr-20Ti-20Al,900,20.0,0.0,20.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.447000,NaN,HEA,NaN,"B. Gorr et al., “High temperature oxidation be...",-0.349692,True
3,2,20Nb-20Mo-20Cr-20Ti-20Al,1000,20.0,0.0,20.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.770000,NaN,HEA,NaN,"B. Gorr et al., “High-Temperature Oxidation Be...",0.442480,True
4,2,20Nb-20Mo-20Cr-20Ti-20Al,1100,20.0,0.0,20.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.380000,NaN,HEA,NaN,"B. Gorr et al., “High-Temperature Oxidation Be...",0.730782,True


In [16]:
all_feats = pd.concat((data, literature_data), axis=0, ignore_index=True)
all_feats['Al'] = all_feats['Al']*10
all_feats['Ni'] = all_feats['Ni']*5

np.random.seed(0)
umap_2d = umap.UMAP(n_components=2, init='random', random_state=0)
proj_2d = umap_2d.fit_transform(all_feats.loc[:,'Al':'Zr'])

all_feats['umap0'] = proj_2d[:,0]
all_feats['umap1'] = proj_2d[:,1]

all_feats['Al'] = all_feats['Al']/10
all_feats['Ni'] = all_feats['Ni']/5

In [6]:
def plot_2d(data, label, marker, lit_check, color_feat, cmin, cmax, size, color):    
    
    if lit_check:
        dataframe = data[data['literature']==True]
                    
        nums = dataframe.shape[0]
        dataframe = dataframe.round({'mass_change':2,
                                     'log_mass_change':2})
        
        # text wrap the long citations
        dataframe['wrap_cit'] = np.nan
        for ndex, row in dataframe.iterrows():
            if row['literature']:
                dataframe.loc[ndex, 'wrap_cit'] = '<br>'.join(textwrap.wrap(row['citation'], 64))
        
        
        text = dataframe[['composition', 'mass_change', 'log_mass_change', 'T (C)', 'wrap_cit']].to_numpy()
        
        if color==False:
            color = dataframe[color_feat]
        else:
            color = color

        fig = go.Scatter(x = dataframe['umap0'],
                          y =  dataframe['umap1'],
                          mode='markers',
                          name= str(nums) + ' ' + label,
                          customdata=text,
                          hovertemplate='composition:%{customdata[0]} <br>'+
                                        'mass_change (mg/cm2): %{customdata[1]}<br>'+
                                        'log(mass change): %{customdata[2]}<br>'+
                                        'exp temperature: %{customdata[3]}<br>'+
                                        'source: %{customdata[4]}<br>',
                          marker=dict(color=color,
                                      size=size,
                                      symbol=marker,
                                      cmin=cmin,
                                      cmax=cmax,
                                      line=dict(width=1,
                                                color='DarkSlateGrey')))
        
    else:
        dataframe = data[~data['literature']]
            
        nums = dataframe.shape[0]
        dataframe = dataframe.round({'solidus':0,
                                     'freeze_range':1,
                                     'max_phase_fraction':2,
                                      'mass_change':2,
                                     'log_mass_change':2})

        text = dataframe[['composition', 'solidus', 'freeze_range',
                          'max_phase', 'max_phase_fraction', 'mass_change', 'log_mass_change']].to_numpy()
        
        if color==False:
            color = dataframe[color_feat]
        else:
            color = color

        fig = go.Scatter(x = dataframe['umap0'],
                          y =  dataframe['umap1'],
                          mode='markers',
                          name= str(nums) + ' ' + label,
                          customdata=text,
                          hovertemplate='composition: %{customdata[0]} <br>'+
                                        'solidus (C): %{customdata[1]} <br>'+
                                        'freeze range: %{customdata[2]} <br>'+
                                        'primary phase: %{customdata[3]} <br>'+
                                        'max phase fraction: %{customdata[4]} <br>'+
                                        'mass_change (mg/cm2): %{customdata[5]}<br>'+
                                        'log(mass change): %{customdata[6]}<br>',
                          marker=dict(color=color,
                                      size=size,
                                      symbol=marker,
                                      cmin=cmin,
                                      cmax=cmax,
                                      line=dict(width=1,
                                                color='DarkSlateGrey')))
    
    return fig

In [15]:
color_feat = 'log_mass_change'
cmin = all_feats[color_feat].min()
cmax = all_feats[color_feat].max()
size = 22
leg_size = 30

# Plot for literature points
data_in = all_feats
fig1 =  plot_2d(data_in, 'Literature Data', 'cross', True, color_feat, cmin, cmax, size, color=False)

# Plot for sampled comps    
fig2 =  plot_2d(data_in, 'New Compositions', 'diamond', False, color_feat, cmin, cmax, size, color=False)

# Plot for chosen comps
compsg = ['Al35 Fe30 Cr15 Mn10 Nb10 ', 'Al40 Fe25 Cr20 Co15 ', 'Co45 Al30 Cr10 Fe10 Ta5 ', 
          'Co35 Al25 Cr20 Mn20 ', 'Ni30 Al25 Mn25 V15 Si5 ']
compsb = ['Ti45 Nb20 Al15 Cr15 Ni5 ', 'Al35 V25 Mn20 Fe10 Nb10 ']

compsg_in = pd.DataFrame()
for comp in compsg+compsb:
    compsg_in = compsg_in.append(data_in[data_in['composition']==comp])

compsb_in = pd.DataFrame()
for comp in compsb:
    compsb_in = compsb_in.append(data_in[data_in['composition']==comp])
    
fig3 = plot_2d(compsg_in, 'Alloys Screened', 'circle', False, color_feat, cmin, cmax, size, color=False)

data_TC_screening = data_in[(data_in['solidus']>1200) & 
                            (data_in['max_phase'].str.startswith('BCC') | data_in['max_phase'].str.startswith('FCC') | data_in['max_phase'].str.startswith('HCP')) &
                            (data_in['max_phase_fraction']>0.7) & 
                            (data_in['freeze_range']<200)]
fig4 = plot_2d(data_TC_screening, 'TC Data Screening', 'triangle-down', False, color_feat, cmin, cmax, size, color=False)

data_LM = data_TC_screening[data_TC_screening['mass_change']<2]
data_HM = data_TC_screening[data_TC_screening['mass_change']>4.4]

fig5 = plot_2d(data_LM, 'LM Alloys', 'triangle-left', False, color_feat, cmin, cmax, size, color=False)
fig6 = plot_2d(data_HM, 'HM Alloys', 'triangle-right', False, color_feat, cmin, cmax, size, color=False)


# plot all data points and format figure
fig = go.Figure(layout={'height':850,'width':1400,'showlegend':True })

fig.add_trace(fig1)
fig.add_trace(fig2)
fig.add_trace(fig4)
fig.add_trace(fig5)
fig.add_trace(fig6)
fig.add_trace(fig3)


# color bar formats
fig.update_traces(marker=dict(showscale=True,
                              reversescale=True, 
                              colorbar=dict(title=dict(text='log10(m/A)',
                                                       side="right", 
                                                       font_size=leg_size),
                                            tickfont=dict(size=leg_size),
                                            thickness=20, 
                                            tickvals= [np.round(cmin, 3), np.round(cmax, 3)]) ))
# increase axis limits and change legend
fig.update_layout(xaxis={'autorange':False,
                         "range":[data_in['umap0'].min()-2,
                                  data_in['umap0'].max()+2]},
                  yaxis={'autorange':False,
                         "range":[data_in['umap1'].min()-2,
                                  data_in['umap1'].max()+2]}, 
                  legend={'x':1.15, 'y':0.5,
                          'itemsizing':'constant',
                          'bgcolor':'white'}, 
                  legend_font_size=leg_size, 
                  plot_bgcolor='white')

# gridline formats
grid_on = False
fig.update_xaxes(showticklabels=grid_on, showgrid=grid_on, gridwidth=1, gridcolor='mediumpurple', zeroline=False)
fig.update_yaxes(showticklabels=grid_on, showgrid=grid_on, gridwidth=1, gridcolor='mediumpurple', zeroline=False)

# add background ovals to highlight clusters
clust_text_size = 18

# 1st cluster
fig.add_trace(go.Scatter(
    x=[1.75],
    y=[21],
    text=["Al30, Al35 and Ni-Al"],
    mode="text",
    textfont=dict(
        color="LightSeaGreen",
        size=clust_text_size,
        #family="Arail",
    )
))
fig.add_shape(type="circle",
    xref="x", yref="y",
    fillcolor="PaleTurquoise",
    x0=-0.5, y0=15, x1=4, y1=20,
    line_color="LightSeaGreen",
    opacity=1, layer="below"
)

# 2nd cluster
fig.add_trace(go.Scatter(
    x=[18],
    y=[19.5],
    text=["Al25 w/ no Ni"],
    mode="text",
    textfont=dict(
        color="LightSeaGreen",
        size=clust_text_size,
        #family="Arail",
    )
))
fig.add_shape(type="circle",
    xref="x", yref="y",
    fillcolor="PaleTurquoise",
    x0=12.5, y0=18, x1=14.75, y1=20.75,
    line_color="LightSeaGreen",
    opacity=1, layer="below"
)

# 3rd cluster
fig.add_trace(go.Scatter(
    x=[-1.25],
    y=[12.5],
    text=["Al5, Al10 and no Ni"],
    mode="text",
    textfont=dict(
        color="LightSeaGreen",
        size=clust_text_size,
        #family="Arail",
    )
))
fig.add_shape(type="circle",
    xref="x", yref="y",
    fillcolor="PaleTurquoise",
    x0=-2, y0=8, x1=0.5, y1=11.5,
    line_color="LightSeaGreen",
    opacity=1, layer="below"
)

# 4th cluster
fig.add_trace(go.Scatter(
    x=[6.25],
    y=[12.25],
    text=["Al<10 and Ni15-45"],
    mode="text",
    textfont=dict(
        color="LightSeaGreen",
        size=clust_text_size,
        #family="Arail",
    )
))
fig.add_shape(type="circle",
    xref="x", yref="y",
    fillcolor="PaleTurquoise",
    x0=2, y0=5, x1=8.5, y1=11.5,
    line_color="LightSeaGreen",
    opacity=1, layer="below"
)

# 5th cluster
fig.add_trace(go.Scatter(
    x=[12.5],
    y=[15],
    text=["Al40, Al45 and Ni<25"],
    mode="text",
    textfont=dict(
        color="LightSeaGreen",
        size=clust_text_size,
        #family="Arail",
    )
))
fig.add_shape(type="circle",
    xref="x", yref="y",
    fillcolor="PaleTurquoise",
    x0=11, y0=11.5, x1=14, y1=14,
    line_color="LightSeaGreen",
    opacity=1, layer="below"
)

# 6th cluster
fig.add_trace(go.Scatter(
    x=[19],
    y=[5.75],
    text=["Al15 and no Ni"],
    mode="text",
    textfont=dict(
        color="LightSeaGreen",
        size=clust_text_size,
        #family="Arail",
    )
))
fig.add_shape(type="circle",
    xref="x", yref="y",
    fillcolor="PaleTurquoise",
    x0=17, y0=2.75, x1=19.5, y1=5,
    line_color="LightSeaGreen",
    opacity=1, layer="below"
)

# 7th cluster
fig.add_trace(go.Scatter(
    x=[-1],
    y=[5.25],
    text=["no Al or Ni"],
    mode="text",
    textfont=dict(
        color="LightSeaGreen",
        size=clust_text_size,
        #family="Arail",
    )
))
fig.add_shape(type="circle",
    xref="x", yref="y",
    fillcolor="PaleTurquoise",
    x0=-4.25, y0=-1.75, x1=2.25, y1=4.75,
    line_color="LightSeaGreen",
    opacity=1, layer="below"
)

# 8th cluster
fig.add_trace(go.Scatter(
    x=[12.5],
    y=[4.25],
    text=["High Ni and Al15-25"],
    mode="text",
    textfont=dict(
        color="LightSeaGreen",
        size=clust_text_size,
        #family="Arail",
    )
))
fig.add_shape(type="circle",
    xref="x", yref="y",
    fillcolor="PaleTurquoise",
    x0=11.5, y0=0.5, x1=14.5, y1=3.5,
    line_color="LightSeaGreen",
    opacity=1, layer="below"
)

# 9th cluster
fig.add_trace(go.Scatter(
    x=[9],
    y=[0],
    text=["Al20 and no Ni"],
    mode="text",
    textfont=dict(
        color="LightSeaGreen",
        size=clust_text_size,
        #family="Arail",
    )
))
fig.add_shape(type="circle",
    xref="x", yref="y",
    fillcolor="PaleTurquoise",
    x0=10.5, y0=-3, x1=13, y1=-0.5,
    line_color="LightSeaGreen",
    opacity=1, layer="below"
)

# 10th cluster
fig.add_trace(go.Scatter(
    x=[7.75],
    y=[-3.9],
    text=["Al45 and no Ni"],
    mode="text",
    textfont=dict(
        color="LightSeaGreen",
        size=clust_text_size,
        #family="Arail",
    )
))
fig.add_shape(type="circle",
    xref="x", yref="y",
    fillcolor="PaleTurquoise",
    x0=9.8, y0=-5.6, x1=11.3, y1=-4.1,
    line_color="LightSeaGreen",
    opacity=1, layer="below"
)

# text for last lit clusters
fig.add_trace(go.Scatter(
    x=[21, 19.6, 15.2, 21.5],
    y=[10.5, -0.6, -4.75, -6],
    text=["Ni alloys", 'Steels', 'Co-Ni alloys', 'Co-Al alloys'],
    mode="text",
    textfont=dict(
        color="LightSeaGreen",
        size=clust_text_size,
        #family="Arail",
    )
))

for trace in fig['data']:
    if trace.mode == 'text':
        trace['showlegend'] = False

fig.show()

In [ ]:
fig.write_html("visualization_update_TC_added.html")